In [1]:
import pandas as pd
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import zipfile

/usr/share/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
def load_faces(path,size):
    faces = np.zeros([size,224,224,3])
    for i in range(size):
        img = image.load_img('{}/{}.jpg'.format(path,i))
        faces[i,:,:] = img
#         if(i%100==0):
#             print("done with {} images".format(i))
    return faces


In [10]:
proportional_train = load_faces('samples/adience_sample_1_train',500)
proportional_val = load_faces('samples/adience_sample_1_val',500)
equal_sub_train = load_faces('samples/adience_sample_2_train',500)
equal_sub_val = load_faces('samples/adience_sample_2_val',500)

done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images
done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images
done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images
done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images


In [17]:
def process_faces(faces):
    l = faces.shape[0]
    processed_faces = np.zeros([l,224,224,3])
    for i in range(l):
        x = np.expand_dims(faces[i], axis = 0)
        x = preprocess_input(x)
        processed_faces[i,:,:] = x
        if(i%100==0):
            print("done with {} images".format(i))
    return processed_faces

In [18]:
prop_train = process_faces(proportional_train)
prop_val = process_faces(proportional_val)
eq_sub_train = process_faces(equal_sub_train)
eq_sub_val = process_faces(equal_sub_val)

done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images
done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images
done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images
done with 0 images
done with 100 images
done with 200 images
done with 300 images
done with 400 images


In [19]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [27]:
# load in metadata
# sample 1: proportional classes (as compared to adience dataset)
sample1_train = pd.read_csv('samples/sample_1_train.csv')
sample1_val = pd.read_csv('samples/sample_1_val.csv')

# sample 2: equal classes
sample2_train = pd.read_csv('samples/sample_2_train.csv')
sample2_val = pd.read_csv('samples/sample_2_val.csv')

In [28]:
y_train_prop = sample1_train['gender_y']
y_val_prop = sample1_val['gender_y']

y_train_eq = sample2_train['gender_y']
y_val_eq = sample2_val['gender_y']

In [29]:
y_train_prop = to_categorical(y_train_prop.map({'f':0,'m':1}),num_classes=2)
y_val_prop = to_categorical(y_val_prop.map({'f':0,'m':1}),num_classes=2)

y_train_eq = to_categorical(y_train_eq.map({'f':0,'m':1}),num_classes=2)
y_val_eq = to_categorical(y_val_eq.map({'f':0,'m':1}),num_classes=2)

## Adience-style network

In [38]:
# model from scratch (mfs)
# using the caffe model params as a guide
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import SGD, Adam

mfs = Sequential()

# conv1
mfs.add(Conv2D(filters = 96, kernel_size = 7, strides = 4,
              activation = 'relu',input_shape = (224,224,3)))
mfs.add(MaxPooling2D(pool_size = 3, strides = 2))

# conv2
mfs.add(Conv2D(filters = 256, kernel_size = 5, padding = 'same',
              activation = 'relu'))
mfs.add(MaxPooling2D(pool_size = 3, strides = 2))

# conv3
mfs.add(Conv2D(filters = 384, kernel_size = 3,
              activation = 'relu'))
mfs.add(MaxPooling2D(pool_size = 3, strides = 2))

mfs.add(Dense(512, activation='relu'))
mfs.add(Dropout(rate=0.5))
mfs.add(Dense(512, activation='relu'))
mfs.add(Dropout(rate=0.5))
mfs.add(Flatten())
mfs.add(Dense(8))
mfs.add(Dense(2,activation='sigmoid'))

# sgd = SGD(lr=.1, decay=0, momentum=0.9, nesterov=True)
mfs.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
mfs.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 55, 55, 96)        14208     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 384)       885120    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 384)         0         
_________________________________________________________________
dense_9 (Dense)              (None, 5, 5, 512)         197120    
__________

In [40]:
mfs.fit(prop_train,y_train_prop,batch_size=128,epochs=25,verbose=1,
        validation_data=(prop_val, y_val_prop))

Train on 500 samples, validate on 500 samples
Epoch 1/25
500/500 [==============================] - 39s 77ms/step - loss: 0.8691 - acc: 0.5190 - val_loss: 0.7079 - val_acc: 0.5000
Epoch 2/25
500/500 [==============================] - 38s 76ms/step - loss: 0.7181 - acc: 0.4970 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 3/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6930 - acc: 0.5210 - val_loss: 0.7083 - val_acc: 0.4570
Epoch 4/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6996 - acc: 0.5120 - val_loss: 0.6893 - val_acc: 0.5490
Epoch 5/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6993 - acc: 0.5360 - val_loss: 0.6889 - val_acc: 0.5220
Epoch 6/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6838 - acc: 0.5690 - val_loss: 0.6845 - val_acc: 0.5810
Epoch 7/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6772 - acc: 0.5780 - val_loss: 0.6787 - val_acc: 0.5870
Epoch 8/25
500/5

In [42]:
mfs.save_weights('mfs_weights_1.h5')

In [43]:
mfs.save('mfs_model_1.h5')

In [46]:
mfs.fit(equal_sub_train,y_train_eq,batch_size=128,epochs=25,verbose=1,
        validation_data=(equal_sub_val, y_val_eq))

Train on 500 samples, validate on 500 samples
Epoch 1/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6923 - acc: 0.6380 - val_loss: 0.6412 - val_acc: 0.6510
Epoch 2/25
500/500 [==============================] - 38s 76ms/step - loss: 0.6285 - acc: 0.6630 - val_loss: 0.6065 - val_acc: 0.6810
Epoch 3/25
500/500 [==============================] - 38s 76ms/step - loss: 0.5732 - acc: 0.6940 - val_loss: 0.5876 - val_acc: 0.6910
Epoch 4/25
500/500 [==============================] - 38s 76ms/step - loss: 0.5389 - acc: 0.7300 - val_loss: 0.5820 - val_acc: 0.6920
Epoch 5/25
500/500 [==============================] - 38s 76ms/step - loss: 0.4927 - acc: 0.7790 - val_loss: 0.5777 - val_acc: 0.7160
Epoch 6/25
500/500 [==============================] - 38s 76ms/step - loss: 0.4729 - acc: 0.7900 - val_loss: 0.5698 - val_acc: 0.7290
Epoch 7/25
500/500 [==============================] - 38s 76ms/step - loss: 0.4017 - acc: 0.8130 - val_loss: 0.5431 - val_acc: 0.7600
Epoch 8/25
500/5

In [47]:
mfs.save_weights('mfs_weights_2.h5')
mfs.save('mfs_model_2.h5')

## VGGFace

In [44]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace

In [45]:
#custom parameters
nb_class = 2
hidden_dim = 512

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)
custom_vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

58916864/58909280 [==============================] - 0s 0us/step


In [ ]:
custom_vgg_model.fit(processed_faces[:100],y_cat[:100],batch_size=64,epochs=100,verbose=1,
        validation_data=(processed_faces[100:125], y_cat[100:125]))

In [ ]:
print("yo")